<a href="https://colab.research.google.com/github/TashinAhmed/Various-Architectures-of-Convolutional-Networks/blob/master/Finetuning_Basenets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Model
From Keras, we can easily use some image classification models.

Xception, VGG16, VGG19, ResNet50, InceptionV3

more on http://marubon-ds.blogspot.com/2017/09/vgg-fine-tuning-model.html

#import libraries

In [0]:
import random
import cv2
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import SGD
from keras.applications.inception_v3 import InceptionV3
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg19 import VGG19
from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception
import numpy as np

Using TensorFlow backend.


#making data

In [0]:
# read data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# limit the amount of the data
# train data
ind_train = random.sample(list(range(x_train.shape[0])), 1000)
x_train = x_train[ind_train]
y_train = y_train[ind_train]

# test data
ind_test = random.sample(list(range(x_test.shape[0])), 1000)
x_test = x_test[ind_test]
y_test = y_test[ind_test]

def resize_data(data):
    data_upscaled = np.zeros((data.shape[0], 320, 320, 3))
    for i, img in enumerate(data):
        large_img = cv2.resize(img, dsize=(320, 320), interpolation=cv2.INTER_CUBIC)
        data_upscaled[i] = large_img

    return data_upscaled

# resize train and  test data
x_train_resized = resize_data(x_train)
x_test_resized = resize_data(x_test)

# make explained variable hot-encoded
y_train_hot_encoded = to_categorical(y_train)
y_test_hot_encoded = to_categorical(y_test)

170500096/170498071 [==============================] - 2s 0us/step


#setting model

In [0]:
def model(x_train, y_train, base_model):

    # get layers and add average pooling layer
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    # add fully-connected layer
    x = Dense(512, activation='relu')(x)
    # add output layer
    predictions = Dense(10, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
# -------------------------------------------------------------------------------
    # freeze pre-trained model area's layer
    for layer in base_model.layers:
        layer.trainable = False

    # update the weight that are added
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
    model.fit(x_train, y_train)
# -------------------------------------------------------------------------------
    # choose the layers which are updated by training
    layer_num = len(model.layers)
    for layer in model.layers[:int(layer_num * 0.9)]:
        layer.trainable = False

    for layer in model.layers[int(layer_num * 0.9):]:
        layer.trainable = True

    # update the weights
    model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(x_train, y_train, epochs=5)
    return history

inception_model = InceptionV3(weights='imagenet', include_top=False)
res_50_model = ResNet50(weights='imagenet', include_top=False)
vgg_19_model = VGG19(weights='imagenet', include_top=False)
vgg_16_model = VGG16(weights='imagenet', include_top=False)
xception_model = Xception(weights='imagenet', include_top=False)

W0806 19:47:51.575990 140395060819840 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0806 19:47:51.623878 140395060819840 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0806 19:47:51.632300 140395060819840 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0806 19:47:51.682504 140395060819840 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0806 19:47:51.683752 1403950608

87916544/87910968 [==============================] - 1s 0us/step


/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


83689472/83683744 [==============================] - 1s 0us/step


#evaluation

In [0]:
history_inception_v3 = model(x_train_resized, y_train_hot_encoded, inception_model)
history_res_50 = model(x_train_resized, y_train_hot_encoded, res_50_model)
history_vgg_19 = model(x_train_resized, y_train_hot_encoded, vgg_19_model)
history_vgg_16 = model(x_train_resized, y_train_hot_encoded, vgg_16_model)
history_xception = model(x_train_resized, y_train_hot_encoded, xception_model)

W0806 19:48:36.038173 140395060819840 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0806 19:48:36.176939 140395060819840 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/1


In [0]:
# check accuracy
evaluation_inception_v3 = history_inception_v3.model.evaluate(x_test_resized,y_test_hot_encoded)
evaluation_res_50 = history_res_50.model.evaluate(x_test_resized,y_test_hot_encoded)
evaluation_vgg_19 = history_vgg_19.model.evaluate(x_test_resized,y_test_hot_encoded)
evaluation_vgg_16 = history_vgg_16.model.evaluate(x_test_resized,y_test_hot_encoded)
evaluation_xception = history_xception.model.evaluate(x_test_resized,y_test_hot_encoded)

print("inception_v3:{}".format(evaluation_inception_v3))
print("res_50:{}".format(evaluation_res_50))
print("vgg_19:{}".format(evaluation_vgg_19))
print("vgg_16:{}".format(evaluation_vgg_16))
print("xception:{}".format(evaluation_xception))



 OUTCOME
 

---


 inception_v3:[1.5071683092117309, 0.622]

 res_50:[1.324682321548462, 0.55100000000000005]
 
 vgg_19:[1.0588858013153075, 0.63600000000000001]
 
 vgg_16:[2.7452965526580813, 0.57899999999999996]
 
 xception:[0.89050176906585699, 0.70999999999999996]